
&nbsp; 
  
#  <center>The Imperfect Umpire Model - Data</center>
## <center>Can we use Statcast data to predict MLB umpire mistakes?</center>
##### <center>University of Colorado Boulder | Introduction to Deep Learning - Final Project</center>
##### <center>Jeremy Hurley | March, 2025</center>

&nbsp;  

---  

&nbsp;  
  

This notebook will import, clean and save a dataset for use in my final project.  
  
See the main notebook *'The Umpire Model'* for the remainder of the project.

# Data Acquisition:  
The pybaseball Python package was developed by James LeDoux. It pulls data from multiple data sources, including FanGraphs, Baseball Reference and Baseball Savant. It contains a vast amount of data from every MLB game played in the modern era. For more information on pybaseball, find it's GitHub repo [here](https://github.com/jldbc/pybaseball)  

This project starts by pulling all statcast data from the 2023 season.

In [1]:
import pybaseball as pb
import pandas as pd

# Pull statcast data for the entire 2023 MLB season
data = pb.statcast(start_dt='2023-03-30', end_dt='2023-10-01')

# Look at first few rows of data.
data.head()

This is a large query, it may take a moment to complete


C:\Users\hurle\OneDrive\MSDS_Colorado\DTSA 550X - CS Pathway\DTSA 5509 - Intro to Machine Learning\FinalProject\env\Lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)
  0%|                                                                                                 | 0/186 [00:00<?, ?it/s]C:\Users\hurle\OneDrive\MSDS_Colorado\DTSA 550X - CS Pathway\DTSA 5509 - Intro to Machine Learn

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,...,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle
2372,CH,2023-10-01,89.0,-2.8,5.59,"Robertson, Nick",677008,687798,field_out,hit_into_play,...,1,3,11,1,<NA>,<NA>,2.55,1.53,-1.53,31.7
2535,FF,2023-10-01,96.9,-2.4,5.9,"Robertson, Nick",677008,687798,NaN,foul,...,1,3,11,1,<NA>,<NA>,1.09,0.76,-0.76,47.4
2631,CH,2023-10-01,90.0,-2.93,5.56,"Robertson, Nick",677008,687798,NaN,ball,...,1,3,11,1,<NA>,<NA>,2.47,1.65,-1.65,30.3
2645,ST,2023-10-01,82.2,-3.09,5.55,"Robertson, Nick",677008,687798,NaN,ball,...,1,3,11,1,<NA>,<NA>,3.14,-1.43,1.43,28.9
2831,CH,2023-10-01,89.2,-2.87,5.58,"Robertson, Nick",677008,687798,NaN,swinging_strike,...,1,3,11,1,<NA>,<NA>,2.57,1.49,-1.49,34.3


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 721486 entries, 2372 to 4486
Columns: 113 entries, pitch_type to arm_angle
dtypes: Float64(35), Int64(61), datetime64[ns](1), object(16)
memory usage: 693.6+ MB


In [3]:
for col in data.columns:
    print(col)

pitch_type
game_date
release_speed
release_pos_x
release_pos_z
player_name
batter
pitcher
events
description
spin_dir
spin_rate_deprecated
break_angle_deprecated
break_length_deprecated
zone
des
game_type
stand
p_throws
home_team
away_team
type
hit_location
bb_type
balls
strikes
game_year
pfx_x
pfx_z
plate_x
plate_z
on_3b
on_2b
on_1b
outs_when_up
inning
inning_topbot
hc_x
hc_y
tfs_deprecated
tfs_zulu_deprecated
umpire
sv_id
vx0
vy0
vz0
ax
ay
az
sz_top
sz_bot
hit_distance_sc
launch_speed
launch_angle
effective_speed
release_spin_rate
release_extension
game_pk
fielder_2
fielder_3
fielder_4
fielder_5
fielder_6
fielder_7
fielder_8
fielder_9
release_pos_y
estimated_ba_using_speedangle
estimated_woba_using_speedangle
woba_value
woba_denom
babip_value
iso_value
launch_speed_angle
at_bat_number
pitch_number
pitch_name
home_score
away_score
bat_score
fld_score
post_away_score
post_home_score
post_bat_score
post_fld_score
if_fielding_alignment
of_fielding_alignment
spin_axis
delta_home_win_exp
d

In [4]:
print(data['umpire'].head())

2372    <NA>
2535    <NA>
2631    <NA>
2645    <NA>
2831    <NA>
Name: umpire, dtype: Int64


---  
  
Pitch level data, as we've downloaded here from statcast does not contain umpire information. Umpire data will be key to our project, so we will have to get umpire data from a different source and merge with this data set. I'll start by cleaning this data.  

---  

The previous two cells show that we have 721,486 rows and 113 columns of data.  
  
Next step is to clean and filter the data.  

---


In [5]:
# get all unique values of the 'description' column to see how pybaseball categorizes the outcome of each pitch
print(data['description'].unique())

['hit_into_play' 'foul' 'ball' 'swinging_strike' 'called_strike'
 'foul_tip' 'blocked_ball' 'swinging_strike_blocked' 'hit_by_pitch'
 'foul_bunt' 'missed_bunt' 'pitchout' 'bunt_foul_tip']


# Data Cleaning:  

As you can see above, statcast data on the entire 2023 season results in a massive amount of data. My approach to cleaning the data will be as follows:  

1. Remove columns that are not relevant to our model, including player and score information.
   
*I will keep situational information like the count and baserunners because I think further work can explore whether or not the strike zone changes based on the count (3-0 versus 0-2) or if there are runners on base. This will be discussed in the Discussion and Conclusions section.*  

2. Remove rows where the outcome of the pitch was anything except for a ball or called_strike.

*We can't judge a pitch to be a ball or strike by whether or not the batter swung at it. This project is meant to analyze how an umpire calls balls and strikes, therefore, we'll need to remove instances like 'swinging_strike', 'foul', ect.*  

3. Remove rows with missing values in columns that will be key to our model.

*Will start with 'plate_x', 'plate_z' and 'type', then analyze the state of other columns to see if further action is needed.*  

Finally, considering the size of the initial dataset and the amount of time it took to download, I'll save the cleaned data set to a .csv file for future use.

In [33]:
# Filter data for relevant columns
cleaned_data = data[['home_team', 'away_team', 'game_date','player_name', 'batter', 'pitcher', 'plate_x', 'plate_z', 'pitch_type', 'zone', 'sz_top', 'sz_bot', 'type', 'release_speed', 'release_pos_x', 'release_pos_z', 'release_pos_y', 'description', 'stand', 'p_throws', 'balls', 'strikes', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b', 'outs_when_up', 'pitch_name']]

# Look at cleaned data 
cleaned_data.head()

,home_team,away_team,game_date,player_name,batter,pitcher,plate_x,plate_z,pitch_type,zone,...,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,pitch_name
2372,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,0.33,2.01,CH,9,...,2,-1.53,0.33,0.33,2.01,<NA>,<NA>,<NA>,2,Changeup
2535,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,0.09,2.71,FF,5,...,2,-0.76,1.36,0.09,2.71,<NA>,<NA>,<NA>,2,4-Seam Fastball
2631,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,-0.24,0.53,CH,13,...,2,-1.65,0.36,-0.24,0.53,<NA>,<NA>,<NA>,2,Changeup
2645,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,0.81,0.49,ST,14,...,2,1.43,0.28,0.81,0.49,<NA>,<NA>,<NA>,2,Sweeper
2831,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,-0.78,2.05,CH,7,...,1,-1.49,0.31,-0.78,2.05,<NA>,<NA>,<NA>,2,Changeup


In [34]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 721486 entries, 2372 to 4486
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   home_team      721486 non-null  object        
 1   away_team      721486 non-null  object        
 2   game_date      721486 non-null  datetime64[ns]
 3   player_name    721486 non-null  object        
 4   batter         721486 non-null  Int64         
 5   pitcher        721486 non-null  Int64         
 6   plate_x        721215 non-null  Float64       
 7   plate_z        721215 non-null  Float64       
 8   pitch_type     721215 non-null  object        
 9   zone           721215 non-null  Int64         
 10  sz_top         721215 non-null  Float64       
 11  sz_bot         721215 non-null  Float64       
 12  type           721486 non-null  object        
 13  release_speed  721214 non-null  Float64       
 14  release_pos_x  721215 non-null  Float64       
 15  rele

In [35]:
# Filter out rows where the outcome was not a called ball or called strike
cleaned_data = cleaned_data[cleaned_data['description'].isin(['ball', 'called_strike'])]
#Encode 'type' column as numeric: 1 for strike, 0 for balls
cleaned_data['type'] = cleaned_data['type'].apply(lambda x: 1 if x == 'S' else 0)

# Check the data
cleaned_data.head()

,home_team,away_team,game_date,player_name,batter,pitcher,plate_x,plate_z,pitch_type,zone,...,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,pitch_name
2631,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,-0.24,0.53,CH,13,...,2,-1.65,0.36,-0.24,0.53,<NA>,<NA>,<NA>,2,Changeup
2645,BAL,BOS,2023-10-01,"Robertson, Nick",677008,687798,0.81,0.49,ST,14,...,2,1.43,0.28,0.81,0.49,<NA>,<NA>,<NA>,2,Sweeper
3292,BAL,BOS,2023-10-01,"Robertson, Nick",602104,687798,-1.84,4.08,ST,11,...,2,1.66,-0.2,-1.84,4.08,<NA>,<NA>,<NA>,1,Sweeper
3468,BAL,BOS,2023-10-01,"Robertson, Nick",602104,687798,1.13,0.97,ST,14,...,2,1.16,0.01,1.13,0.97,<NA>,<NA>,<NA>,1,Sweeper
3516,BAL,BOS,2023-10-01,"Robertson, Nick",602104,687798,-0.65,2.34,ST,4,...,1,1.45,0.06,-0.65,2.34,<NA>,<NA>,<NA>,1,Sweeper


In [36]:
# Remove rows with missing critical info
cleaned_data = cleaned_data.dropna(subset=['plate_x', 'plate_z', 'type'])
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362022 entries, 2631 to 4486
Data columns (total 31 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   home_team      362022 non-null  object        
 1   away_team      362022 non-null  object        
 2   game_date      362022 non-null  datetime64[ns]
 3   player_name    362022 non-null  object        
 4   batter         362022 non-null  Int64         
 5   pitcher        362022 non-null  Int64         
 6   plate_x        362022 non-null  Float64       
 7   plate_z        362022 non-null  Float64       
 8   pitch_type     362022 non-null  object        
 9   zone           362022 non-null  Int64         
 10  sz_top         362022 non-null  Float64       
 11  sz_bot         362022 non-null  Float64       
 12  type           362022 non-null  int64         
 13  release_speed  362021 non-null  Float64       
 14  release_pos_x  362022 non-null  Float64       
 15  rele


&nbsp;  
  
---  

As you can see in the above cell, we've cleaned the original 113 columns X 721486 rows worth of data to 31 columns X 362022 rows of data. We've filtered out unneeded columns and rows and removed rows with missing data. Further examination of the .info() above shows that, with the exception of on_base columns, which we would expect to be incomplete, all other rows are complete.  

---  

Next step is to import the home plate umpire for each pitch. I was able to find and download umpire information here: https://umpscorecards.com/games/  



In [37]:
# Read in umpire csv data
umpire_df = pd.read_csv("umpire_games_2023.csv")

umpire_df.head()

,Date,Umpire,Home,Away,R [H],R [A],PC,IC,xIC,CC,xCC,CCAx,Acc,xAcc,AAx,Con,Fav [H],totRI
0,9/30/2023,Angel Hernandez,DET,CLE,8,0,177,7,11.7,170,165.3,4.7,96,93.4,2.7,93.8,-0.36,0.8
1,9/30/2023,Alfonso Marquez,TOR,TB,5,7,160,10,8,150,152,-2,93.8,95,-1.3,95,0.5,1.56
2,9/30/2023,Quinn Wolcott,NYM,PHI,4,3,147,7,10.7,140,136.3,3.7,95.2,92.7,2.5,95.2,0.26,1.08
3,9/30/2023,D.J. Reyburn,NYM,PHI,11,4,151,4,7,147,144,3,97.4,95.4,2,98,-0.16,0.4
4,9/30/2023,Chad Fairchild,PIT,MIA,3,7,158,13,16.4,145,141.6,3.4,91.8,89.6,2.2,90.5,-0.95,2.27


In [38]:
umpire_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 18 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     2354 non-null   object
 1   Umpire   2354 non-null   object
 2   Home     2354 non-null   object
 3   Away     2354 non-null   object
 4   R [H]    2354 non-null   int64 
 5   R [A]    2354 non-null   int64 
 6   PC       2354 non-null   object
 7   IC       2354 non-null   object
 8   xIC      2354 non-null   object
 9   CC       2354 non-null   object
 10  xCC      2354 non-null   object
 11  CCAx     2354 non-null   object
 12  Acc      2354 non-null   object
 13  xAcc     2354 non-null   object
 14  AAx      2354 non-null   object
 15  Con      2354 non-null   object
 16  Fav [H]  2354 non-null   object
 17  totRI    2354 non-null   object
dtypes: int64(2), object(16)
memory usage: 331.2+ KB


In [39]:
# Standardize column names and date types
umpire_df.rename(columns={'Date': 'game_date', 'Home': 'home_team', 'Away': 'away_team', 'Umpire': 'umpire'}, inplace = True)

cleaned_data['game_date'] = pd.to_datetime(cleaned_data['game_date'])
umpire_df['game_date'] = pd.to_datetime(umpire_df['game_date'])

# Further investigation found 34 duplicates
# Drop duplicates to ensure unique game entries
umpire_df = umpire_df.drop_duplicates(subset=['game_date', 'home_team', 'away_team'])

In [40]:
# Merge umpire data into cleaned data
merged_df = cleaned_data.merge(umpire_df[['game_date', 'home_team', 'away_team', 'umpire']], on=['game_date', 'home_team', 'away_team'], how='left')
print(merged_df[['game_date', 'home_team', 'away_team', 'umpire']].head())
print(merged_df.info())

   game_date home_team away_team umpire
0 2023-10-01       BAL       BOS    NaN
1 2023-10-01       BAL       BOS    NaN
2 2023-10-01       BAL       BOS    NaN
3 2023-10-01       BAL       BOS    NaN
4 2023-10-01       BAL       BOS    NaN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362022 entries, 0 to 362021
Data columns (total 32 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   home_team      362022 non-null  object        
 1   away_team      362022 non-null  object        
 2   game_date      362022 non-null  datetime64[ns]
 3   player_name    362022 non-null  object        
 4   batter         362022 non-null  Int64         
 5   pitcher        362022 non-null  Int64         
 6   plate_x        362022 non-null  Float64       
 7   plate_z        362022 non-null  Float64       
 8   pitch_type     362022 non-null  object        
 9   zone           362022 non-null  Int64         
 10  sz_top         36202

In [41]:
# Umpire data was pulled for only regular season games. Statcast data was pulled through 10/1, which likely includes some playoff games.
print(merged_df['umpire'].isna().sum())

11543


In [42]:
# Therefore, will drop rows that do not have umpire data
merged_df = merged_df.dropna(subset=['umpire'])
print(merged_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 350479 entries, 2193 to 362021
Data columns (total 32 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   home_team      350479 non-null  object        
 1   away_team      350479 non-null  object        
 2   game_date      350479 non-null  datetime64[ns]
 3   player_name    350479 non-null  object        
 4   batter         350479 non-null  Int64         
 5   pitcher        350479 non-null  Int64         
 6   plate_x        350479 non-null  Float64       
 7   plate_z        350479 non-null  Float64       
 8   pitch_type     350479 non-null  object        
 9   zone           350479 non-null  Int64         
 10  sz_top         350479 non-null  Float64       
 11  sz_bot         350479 non-null  Float64       
 12  type           350479 non-null  int64         
 13  release_speed  350478 non-null  Float64       
 14  release_pos_x  350479 non-null  Float64       
 15  re

In [43]:
print(merged_df[['game_date', 'home_team', 'away_team', 'umpire']].sample(10))

        game_date home_team away_team            umpire
344961 2023-04-08        AZ       LAD      Doug Eddings
86658  2023-08-19       TEX       MIL      Tripp Gibson
103284 2023-08-11        TB       CLE  Junior Valentine
171494 2023-07-03        SF       SEA     Lance Barrett
114669 2023-08-05       CIN       WSH        John Libka
126047 2023-07-30       TOR       LAA     Erich Bacchus
110980 2023-08-07       CLE       TOR    Chad Fairchild
122611 2023-08-01       STL       MIN           Ben May
361215 2023-03-30       CIN       PIT       Mark Wegner
81257  2023-08-22       PHI        SF      Nick Mahrley


In [44]:
# A quick check of 10 sample rows against the original umpire csv file shows the import worked as expected. 
# We can now save our data for use in the final project.
# Save the dataset to a CSV file
merged_df.to_csv('2023_strike_ball_data_with_umpires.csv', index=False)